In [1]:
import sys, os
import scanpy as sc
import numpy as np
import pandas as pd
from scipy import sparse
import matplotlib.pyplot as plt
import anndata as ad
from sklearn.preprocessing import LabelEncoder
from umap import UMAP
import random
le = LabelEncoder()

## Generate simulated doublets based on cell type

In [2]:
def generate_doublet_pairs(ct_pairs, amount, ct_idx, total_cells, sc_data, anno, p_values,CT_info, HOMOTYPIC=False):
    print('Start Simulating...')
    # obtain percentage
    if "%" in amount:
        percent = (float(amount.split('%')[0]) / 100)
        N = round(percent * total_cells/(1-percent))
        print('Origin: ',percent * total_cells, 'Corrected:', N )
    else :
        N = int(amount)
    print('Doublet number to be created: {}'.format(N))

    # ! if ct_pairs == None, then just random choose two ct for each doublet
    if ct_pairs == None:
        sp_ct_lst = list(ct_idx.keys())

    else:
        # Check cell type input    
        assert len(ct_pairs) >= 2, 'At least two cell types are required'
        sp_ct_lst = ct_pairs

    small_ct_pc_lst, large_ct_pc_lst = [], []
    ct_pair_lst_small, ct_pair_lst_large = [], []
    idx_pairs = []
    dbl_fake_ct = []
    # random sample two cells
    for i in range(N):
        ct_1, ct_2 = np.random.choice(sp_ct_lst,
        p=p_values,
        size=2, replace=HOMOTYPIC)
        a = np.random.choice(ct_idx[ct_1], 1)
        b = np.random.choice(ct_idx[ct_2], 1)
        idx_pairs.append((int(a), int(b)))
        # print(ct_1, ct_2)
        ct_1_percent = float(CT_info[ct_1][1][:-1])
        ct_2_percent = float(CT_info[ct_2][1][:-1])
        if ct_1_percent > ct_2_percent:
            ct_pair_lst_large.append(ct_1)
            ct_pair_lst_small.append(ct_2)
            large_ct_pc_lst.append(ct_1_percent)
            small_ct_pc_lst.append(ct_2_percent)
        else:
            ct_pair_lst_small.append(ct_1)
            ct_pair_lst_large.append(ct_2)
            small_ct_pc_lst.append(ct_1_percent)
            large_ct_pc_lst.append(ct_2_percent)
        if random.random() > 0.5:
            dbl_fake_ct.append(ct_1)
        else:
            dbl_fake_ct.append(ct_2)

    # check how many repeatative pairs in sampled pairs
    print('Original {} pairs, unique {} pairs, repeatative {} pairs'.format(len(idx_pairs), len(list(set(idx_pairs))), len(idx_pairs)-len(list(set(idx_pairs)))))
    idx_pairs = np.array(idx_pairs)

    # sample & concate new doublets behind original sc_data
    print('Mtx size before : {}'.format(sc_data.shape))
    new_dbls = []
    for i in idx_pairs:
        sp_1, sp_2 = sc_data[i]
        #! No average
        this_new = (sp_1 + sp_2)
        # random round up / down
        rand = np.random.rand(np.array(this_new).squeeze().shape[0]) - 0.5
        this_new = np.round(np.array(this_new).squeeze() + rand)
        new_dbls.append(np.array(this_new).squeeze())
    # generate new matrix containing artificial doublets
    new_data = np.concatenate((sc_data, np.matrix(np.array(new_dbls))))
    print("Mtx size after: {}".format(new_data.shape))
    # generate new annos for downstream Anndata
    new_anno = anno.append(pd.Series(['doublet'] * N))
    new_fake_anno = anno.append(pd.Series(dbl_fake_ct))
    # print(anno.shape, len( [-1 * anno.shape[0]] + small_ct_pc_lst), new_data.shape)
    doublet_anno = pd.DataFrame({'small_percent': small_ct_pc_lst, 'large_percent': large_ct_pc_lst,
                                  'small_ct': ct_pair_lst_small, 'large_ct': ct_pair_lst_large})

    new_anno_bin = pd.Series(['singlet'] * sc_data.shape[0]).append(pd.Series(['doublet'] * N))
    print('Doublet Percentage {} finished! \n'.format(amount))
    return new_data, new_anno, new_anno_bin, doublet_anno, new_fake_anno

## PBMC data

In [6]:
# Using PBMC data from scanpy
adata = sc.read_10x_mtx(
    './pbmc3k_filtered_gene_bc_matrices/filtered_gene_bc_matrices/hg19/',  # the path to the `.mtx` file
    var_names='gene_symbols',
    cache=True)            

adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`

sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]
adata1 = sc.read_10x_mtx(
    './pbmc3k_filtered_gene_bc_matrices/filtered_gene_bc_matrices/hg19/',
    var_names='gene_symbols',         
    cache=True)
# adata1 = adata1[adata1.obs.index.isin(adata.obs.index)]
adata1 = adata1[adata.obs.index, :]
print(adata1)
meta_2 = pd.read_csv('./pbmc3k_filtered_gene_bc_matrices/filtered_gene_bc_matrices/hg19/meta_pbmc_2638_full.csv')
data_kdy2 = np.matrix(adata1.X.toarray())
print(data_kdy2.shape, meta_2.shape)
print(meta_2['cell_types'].value_counts().shape)

percent = ["%.3f%%" % (i * 100) for i in (meta_2['cell_types'].value_counts()/data_kdy2.shape[0])]
cnt = np.array(meta_2['cell_types'].value_counts())
CT_info = pd.Series(list(zip(cnt, percent)), index=meta_2['cell_types'].value_counts().index)
ct_lst = np.array(meta_2['cell_types'].value_counts().index)
total_cells = data_kdy2.shape[0]
ct_idx = {}
for i in ct_lst:
    this_idx = meta_2[meta_2['cell_types'] == i].index
    ct_idx[i] = np.array(this_idx)
np.set_printoptions(precision=10)
p_values = (np.array(list(zip(*CT_info.values))[0])/total_cells ).astype('float64')
CT_info

View of AnnData object with n_obs × n_vars = 2638 × 32738
    var: 'gene_ids'
(2638, 32738) (2638, 2)
(8,)


0    (1135, 43.025%)
1     (431, 16.338%)
2     (341, 12.926%)
3     (313, 11.865%)
4      (205, 7.771%)
5      (164, 6.217%)
6       (36, 1.365%)
7       (13, 0.493%)
dtype: object

In [8]:
# For PBMC
random.seed(123)
store_dir = 'simulated/' # The dir to save simulated data
root_path = './pbmc3k_filtered_gene_bc_matrices/' # The root path of the store directory
if not os.path.exists(root_path+store_dir):
    os.mkdir(root_path+store_dir)
for i in [*[str(i)+'%' for i in range(1, 41)]]:
    simulate_data, anno_data, anno_bin_data, anno_doublet_data, anno_fake = generate_doublet_pairs(ct_pairs=None, amount=i, 
    ct_idx=ct_idx ,total_cells=total_cells, CT_info=CT_info,
    sc_data=data_kdy2, anno=meta_2['cell_types'], p_values=p_values,
    HOMOTYPIC=False)
    # simulate_data, anno_data, anno_bin_data = generate_doublet_pairs(ct_pairs=['NK', 'Podo'], amount=i, ct_idx=ct_idx ,total_cells=total_cells, sc_data=data_kdy2, anno=meta_2['cell_types'])
    
    df = pd.DataFrame(simulate_data)
    df_anno = pd.DataFrame(anno_data)
    df_anno_bin = pd.DataFrame(anno_bin_data)
    df_anno_doublet = pd.DataFrame(anno_doublet_data)
    df_anno_doublet_fake = pd.DataFrame(anno_fake)

    sparse.save_npz(root_path+store_dir+'/sc_data_{}.npz'.format(i.split('%')[0]), sparse.coo_matrix(df))
    df.to_csv((root_path+store_dir+'/sc_data_{}.csv'.format(i.split('%')[0])))
    df_anno.to_csv(root_path+store_dir+'/meta_{}.csv'.format(i.split('%')[0]))
    df_anno_bin.to_csv(root_path+store_dir+'/meta_bin_{}.csv'.format(i.split('%')[0]))
    df_anno_doublet.to_csv(root_path+store_dir+'/doublet_anno_{}.csv'.format(i.split('%')[0]))
    df_anno_doublet_fake.to_csv(root_path+store_dir+'/doublet_anno_fake_{}.csv'.format(i.split('%')[0]))

    print('simulated_{} finished!'.format(i.split('%')[0]))

Start Simulating...
Origin:  26.38 Corrected: 27
Doublet number to be created: 27
Original 27 pairs, unique 27 pairs, repeatative 0 pairs
Mtx size before : (2638, 32738)
Mtx size after: (2665, 32738)
Doublet Percentage 1% finished! 



C:\Users\xzhao429\AppData\Local\Temp\ipykernel_13224\992307633.py:79: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_anno = anno.append(pd.Series(['doublet'] * N))
C:\Users\xzhao429\AppData\Local\Temp\ipykernel_13224\992307633.py:80: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_fake_anno = anno.append(pd.Series(dbl_fake_ct))
C:\Users\xzhao429\AppData\Local\Temp\ipykernel_13224\992307633.py:85: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_anno_bin = pd.Series(['singlet'] * sc_data.shape[0]).append(pd.Series(['doublet'] * N))


simulated_1 finished!
Start Simulating...
Origin:  52.76 Corrected: 54
Doublet number to be created: 54
Original 54 pairs, unique 54 pairs, repeatative 0 pairs
Mtx size before : (2638, 32738)
Mtx size after: (2692, 32738)
Doublet Percentage 2% finished! 

simulated_2 finished!
Start Simulating...
Origin:  79.14 Corrected: 82
Doublet number to be created: 82
Original 82 pairs, unique 82 pairs, repeatative 0 pairs
Mtx size before : (2638, 32738)
Mtx size after: (2720, 32738)
Doublet Percentage 3% finished! 

simulated_3 finished!
Start Simulating...
Origin:  105.52 Corrected: 110
Doublet number to be created: 110
Original 110 pairs, unique 110 pairs, repeatative 0 pairs
Mtx size before : (2638, 32738)
Mtx size after: (2748, 32738)
Doublet Percentage 4% finished! 

simulated_4 finished!
Start Simulating...
Origin:  131.9 Corrected: 139
Doublet number to be created: 139
Original 139 pairs, unique 139 pairs, repeatative 0 pairs
Mtx size before : (2638, 32738)
Mtx size after: (2777, 32738)
D